In [1]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../data/'
folder = 'sp_matrix_tag_pop'
group_type = 'random'
group_dir_ = '../data/groups/' + group_type
group_info = 'group_info/' + group_type
file_name = 'normalized_to_rating_filter_track_5_user_50.pkl'

In [2]:
df = pd.read_pickle(os.path.join(dir_, folder, group_info, file_name))
df[:5]

,uid,tid,count,tags,rating,pop,group_info_2,group_info_3,group_info_4,group_info_5
0,0,0,2,1,1.50,5,67,8.0,34.0,23.0
2,0,1,2,2,1.50,10,67,8.0,34.0,23.0
7,0,2,4,0,3.00,24,67,8.0,34.0,23.0
8,0,3,1,0,0.25,9,67,8.0,34.0,23.0
10,0,4,5,1,3.50,14,67,8.0,34.0,23.0


In [3]:
groups_n =[]
group_sizes = ['2', '3', '4', '5']
group2 = {}
group3 = {}
group4 = {}
group5 = {}
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)
    j = 0;
    for group in groups:
        for user in group:
            if len(group) == 2:
                group2[user] = j
            elif len(group) == 3:
                group3[user] = j
            elif len(group) == 4:
                group4[user] = j
            elif len(group) == 5:
                group5[user] = j
        j += 1

In [4]:
predictions2 = pd.read_pickle(os.path.join(dir_, folder, group_info, '2_topN_pred_' + file_name[:-3] + 'pkl'))
predictions3 = pd.read_pickle(os.path.join(dir_, folder, group_info, '3_topN_pred_' + file_name[:-3] + 'pkl'))
predictions4 = pd.read_pickle(os.path.join(dir_, folder, group_info, '4_topN_pred_' + file_name[:-3] + 'pkl'))
predictions5 = pd.read_pickle(os.path.join(dir_, folder, group_info, '5_topN_pred_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, folder, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,count,tags,rating,pop
7,0,2,4,0,3.00,24
24,0,14,2,1,1.50,6
25,0,15,12,0,4.75,53
33,0,20,3,12,2.50,12
34,0,21,1,5,0.25,6


In [5]:
predictions2[:5]

,uid,tid,rating
0,0,2,0.075811
1,0,14,0.026264
2,0,15,0.156494
3,0,20,0.043248
4,0,21,0.026327


In [6]:
predictions2['gid'] = predictions2['uid'].map(group2)
predictions3['gid'] = predictions3['uid'].map(group3)
predictions4['gid'] = predictions4['uid'].map(group4)
predictions5['gid'] = predictions5['uid'].map(group5)

In [7]:
predictions5[:5]

,uid,tid,rating,gid
0,0,2,0.075902,23.0
1,0,14,0.026029,23.0
2,0,15,0.155866,23.0
3,0,20,0.042689,23.0
4,0,21,0.026346,23.0


In [8]:
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,count,tags,rating,pop
7,0,2,4,0,3.00,24
24,0,14,2,1,1.50,6
25,0,15,12,0,4.75,53
33,0,20,3,12,2.50,12
34,0,21,1,5,0.25,6


In [9]:
groups_n =[]
group_sizes = ['2', '3', '4', '5']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)
len(groups_n[0][0]), len(groups_n[1][0]), len(groups_n[2][0]), len(groups_n[3][0])

(2, 3, 4, 5)

In [10]:
top_4_results = []
top_20_results = []
top_100_results = []

In [11]:
top_n_size = 100
top_n_lists_g = []
for groups in tqdm(groups_n):
    
    top_n_lists = []

    if len(groups[0]) == 2:
        predictions = predictions2
    elif len(groups[0]) == 3:
        predictions = predictions3
    elif len(groups[0]) == 4:
        predictions = predictions4
    else:
        predictions = predictions5

    for gid in range(len(groups)):
        
        rating_table = predictions[predictions['gid']==gid]
        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        print(rating_table)
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([100, 20, 4]):
    avg_precisions = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            high_rating = 0

            for k in range(top_n_size):
                for l in range(len(group)):
                    uid = group[l]
                    tid = top_n_list[k]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_rating += 1
                        break

            precision = high_rating / top_n_size
            precisions.append(precision)

        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        avg_precisions.append(avg_precision)
    if top_n_size == 4:
        top_4_results.append(avg_precisions)
    elif top_n_size == 20:
        top_20_results.append(avg_precisions)
    elif top_n_size == 100:
        top_100_results.append(avg_precisions)

         uid   tid    rating  gid
1391465   37    55  0.258267    0
1400326   37  9147  0.257603    0
1088620   29    55  0.257255    0
1097658   29  9147  0.256592    0
1397433   37  6190  0.252577    0
...      ...   ...       ...  ...
1392074   37   682  0.202466    0
1090185   29  1651  0.202347    0
1092981   29  4466  0.202308    0
1097789   29  9279  0.202102    0
1089963   29  1427  0.202020    0

[100 rows x 4 columns]
         uid   tid    rating  gid
4437263  118  9147  0.258161    1
6525139  174    55  0.256753    1
6531218  174  6190  0.251092    1
4429965  118  1388  0.247260    1
6525628  174   555  0.245887    1
...      ...   ...       ...  ...
4428982  118   293  0.192018    1
6531246  174  6218  0.190993    1
6526450  174  1390  0.190788    1
6531146  174  6118  0.190681    1
6530301  174  5269  0.190592    1

[100 rows x 4 columns]
         uid   tid    rating  gid
8102438  216  9147  0.257526    2
8093917  216   555  0.247270    2
8094746  216  1388  0.246648    2


         uid   tid    rating  gid
671400    18    55  0.256729   24
680396    18  9147  0.256067   24
1011419   27    55  0.255403   24
1020473   27  9147  0.254740   24
677468    18  6190  0.251070   24
...      ...   ...       ...  ...
680527    18  9279  0.201685   24
672764    18  1427  0.201603   24
672022    18   682  0.201226   24
1013007   27  1651  0.200867   24
1015804   27  4466  0.200827   24

[100 rows x 4 columns]
         uid   tid    rating  gid
7530558  201  9147  0.257945   25
7077387  189   555  0.247939   25
7522165  201   555  0.247673   25
7078181  189  1388  0.247317   25
7522991  201  1388  0.247052   25
...      ...   ...       ...  ...
7077394  189   562  0.198069   25
7527427  201  5920  0.197934   25
7522172  201   562  0.197854   25
7084026  189  7451  0.197849   25
7528908  201  7451  0.197634   25

[100 rows x 4 columns]
         uid    tid    rating  gid
5453612  145     55  0.255259   26
5462580  145   9147  0.254597   26
5459661  145   6190  0.249628  

         uid   tid    rating  gid
3980726  106    55  0.258662   48
3989600  106  9147  0.257998   48
3381255   90  9147  0.257969   48
3986685  106  6190  0.252965   48
3378395   90  6190  0.252937   48
...      ...   ...       ...  ...
3373427   90   922  0.197448   48
3985675  106  5160  0.195231   48
3377402   90  5160  0.195206   48
3985550  106  5035  0.195158   48
3377281   90  5035  0.195133   48

[100 rows x 4 columns]
         uid    tid    rating  gid
6601730  176     55  0.259135   49
2105688   56     55  0.258717   49
2114484   56   9147  0.258054   49
6607426  176   6190  0.253429   49
2106138   56    555  0.247778   49
...      ...    ...       ...  ...
2105898   56    293  0.191932   49
2107302   56   1831  0.191782   49
2106769   56   1236  0.191210   49
6604057  176   2654  0.189660   49
6611717  176  10687  0.189578   49

[100 rows x 4 columns]
         uid   tid    rating  gid
6497186  173  9147  0.258519   50
2919373   78    55  0.258306   50
6489678  173   555  0.

         uid    tid    rating  gid
2770955   74     55  0.259304   72
2778457   74   9147  0.258640   72
2771394   74    555  0.248342   72
6451977  172    555  0.248156   72
6452765  172   1388  0.247534   72
...      ...    ...       ...  ...
6461314  172  10669  0.187359   72
6452681  172   1302  0.187235   72
2771508   74    686  0.187199   72
2777489   74   7882  0.187178   72
6452102  172    686  0.187049   72

[100 rows x 4 columns]
         uid   tid    rating  gid
3067939   82    55  0.258747   73
5739115  153    55  0.255032   73
5748187  153  9147  0.254370   73
5745230  153  6190  0.249404   73
3069173   82  1388  0.247185   73
...      ...   ...       ...  ...
5740755  153  1695  0.191527   73
3069035   82  1236  0.191234   73
5745258  153  6218  0.189657   73
5740450  153  1390  0.189452   73
3070374   82  2654  0.189369   73

[100 rows x 4 columns]
         uid   tid    rating  gid
6389610  170  9147  0.258630   74
6386991  170  6190  0.253585   74
6381862  170   555  0.

         uid   tid    rating  gid
6304708  168    55  0.258315   96
3105389   83    55  0.257838   96
6313701  168  9147  0.257652   96
3114313   83  9147  0.257174   96
6310751  168  6190  0.252625   96
...      ...   ...       ...  ...
3106363   83  1043  0.204648   96
6306294  168  1651  0.203214   96
6309048  168  4466  0.203175   96
6313833  168  9279  0.202968   96
6306072  168  1427  0.202887   96

[100 rows x 4 columns]
         uid    tid    rating  gid
1991350   53     55  0.256870   97
2000383   53   9147  0.256207   97
1997447   53   6190  0.251207   97
1991849   53    555  0.245999   97
1992679   53   1388  0.245379   97
...      ...    ...       ...  ...
1470288   39   2654  0.189766   97
1476480   39   9975  0.189466   97
1470178   39   2542  0.189365   97
1993941   53   2654  0.187928   97
2001914   53  10687  0.187846   97

[100 rows x 4 columns]
         uid    tid    rating  gid
3679488   98     55  0.258553   98
3688421   98   9147  0.257890   98
3685513   98   6190

...

         uid    tid    rating   gid
5552927  148   3206  0.258423  49.0
5150496  137   1772  0.257598  49.0
5159423  137  11080  0.255967  49.0
5558820  148   9332  0.252510  49.0
5156502  137   8033  0.251700  49.0
...      ...    ...       ...   ...
5554428  148   4752  0.202742  49.0
5557149  148   7585  0.202546  49.0
5151097  137   2415  0.202369  49.0
5561777  148  12381  0.202339  49.0
5151823  137   3179  0.202254  49.0

[100 rows x 4 columns]
         uid    tid    rating   gid
7041945  188   3023  0.258749  50.0
3299088   87  38942  0.258704  50.0
6010548  160   3431  0.258249  50.0
3272019   87  11614  0.257366  50.0
6019505  160  12512  0.256617  50.0
...      ...    ...       ...   ...
7042409  188   3493  0.211425  50.0
6011109  160   4007  0.211002  50.0
3270226   87   9780  0.209274  50.0
3266679   87   6114  0.209152  50.0
7044635  188   5748  0.208948  50.0

[100 rows x 4 columns]
         uid    tid    rating   gid
3567380   95   2913  0.258411  51.0
4555422  121  11

         uid    tid    rating   gid
1320128   35   6190  0.252597  69.0
6382573  170   1323  0.247785  69.0
1315445   35   1388  0.247263  69.0
6416538  170  38871  0.246207  69.0
6384622  170   3550  0.245073  69.0
...      ...    ...       ...   ...
6381846  170    536  0.197724  69.0
1323219   35   9395  0.197670  69.0
1314998   35    922  0.197473  69.0
1321334   35   7451  0.197219  69.0
1319868   35   5920  0.197200  69.0

[100 rows x 4 columns]
         uid    tid    rating   gid
745168    20     55  0.258856  70.0
3345849   89  11901  0.256938  70.0
1924744   51   9147  0.256541  70.0
750957    20   6190  0.252934  70.0
3342989   89   8923  0.252653  70.0
...      ...    ...       ...   ...
746018    20    944  0.207999  70.0
1916657   51    944  0.207430  70.0
750432    20   5638  0.206247  70.0
746339    20   1290  0.205963  70.0
745950    20    872  0.205754  70.0

[100 rows x 4 columns]
         uid    tid    rating   gid
3720406   99   2422  0.259011  71.0
5464865  145  11

         uid    tid    rating   gid
782806    21     55  0.257799  23.0
5309640  141   9147  0.257796  23.0
5262360  140     55  0.257475  23.0
791702    21   9147  0.257177  23.0
7377647  197   8465  0.256863  23.0
...      ...    ...       ...   ...
5272139  140   9938  0.213625  23.0
783059    21    321  0.213613  23.0
783091    21    353  0.213591  23.0
5305812  141   5015  0.213523  23.0
7379103  197  10004  0.213440  23.0

[100 rows x 4 columns]
         uid   tid    rating   gid
2105688   56    55  0.258224  24.0
2114484   56  9147  0.257602  24.0
7977435  213   601  0.256991  24.0
7986408  213  9726  0.256370  24.0
7983482  213  6759  0.251511  24.0
...      ...   ...       ...   ...
7978633  213  1850  0.204907  24.0
7978398  213  1609  0.204416  24.0
7985184  213  8492  0.204144  24.0
3945091  105   682  0.203401  24.0
3952859  105  9279  0.203264  24.0

[100 rows x 4 columns]
         uid   tid    rating   gid
2735550   73    55  0.258553  25.0
595282    16    55  0.258060  

         uid   tid    rating   gid
3868413  103    55  0.258448  50.0
5588333  149    55  0.258274  50.0
1730503   46    55  0.258257  50.0
4475248  119  9147  0.257515  50.0
3874022  103  6190  0.252941  50.0
...      ...   ...       ...   ...
1735113   46  4771  0.216910  50.0
4471013  119  4771  0.216806  50.0
3868548  103   199  0.216432  50.0
5588466  149   199  0.216283  50.0
1730643   46   199  0.216266  50.0

[100 rows x 4 columns]
         uid   tid    rating   gid
6195374  165    55  0.257937  51.0
1691964   45    55  0.257931  51.0
6204322  165  9147  0.257315  51.0
1700882   45  9147  0.257309  51.0
1011419   27    55  0.256831  51.0
...      ...   ...       ...   ...
6195518  165   199  0.215991  51.0
1692106   45   199  0.215989  51.0
1014774   27  3436  0.215801  51.0
1016109   27  4771  0.215683  51.0
1011563   27   199  0.215040  51.0

[100 rows x 4 columns]
         uid    tid    rating   gid
3410613   91     55  0.258023  52.0
3419341   91   9147  0.257401  52.0
5385

         uid    tid    rating   gid
6421873  171   6162  0.258159  19.0
2965136   79   8328  0.258131  19.0
2993277   79  38387  0.258035  19.0
7196902  192   6707  0.257956  19.0
3185238   85   2147  0.257604  19.0
...      ...    ...       ...   ...
2993332   79  38442  0.220164  19.0
3185302   85   2211  0.219793  19.0
7227522  192  38443  0.219500  19.0
2554579   68    103  0.219381  19.0
2958785   79   1245  0.219376  19.0

[100 rows x 4 columns]
         uid    tid    rating   gid
7294457  194  32136  0.258120  20.0
745168    20     55  0.258117  20.0
7265930  194   2595  0.258047  20.0
6817442  182   1331  0.257920  20.0
6826076  182  10567  0.257898  20.0
...      ...    ...       ...   ...
4700508  125   9238  0.218584  20.0
4699096  125   7798  0.218569  20.0
4692249  125    560  0.218468  20.0
4702842  125  11620  0.218356  20.0
748375    20   3436  0.217040  20.0

[100 rows x 4 columns]
         uid    tid    rating   gid
2377320   63  10355  0.257821  21.0
1844623   49    

In [12]:
for i in top_4_results:
    print(i)
print('------------------')

for i in top_20_results:
    print(i)
print('------------------')

for i in top_100_results:
    print(i)

[0.04357798165137615, 0.052083333333333336, 0.06481481481481481, 0.0755813953488372]
------------------
[0.04541284403669725, 0.056944444444444436, 0.05092592592592594, 0.06162790697674418]
------------------
[0.04403669724770641, 0.06055555555555556, 0.04666666666666667, 0.06348837209302327]


In [13]:
top_n_size = 100
top_n_lists_g = []
for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_lists = []

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 

        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([100, 20, 4]):
    avg_precisions = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            high_rating = 0

            for k in range(top_n_size):
                for l in range(len(group)):
                    uid = group[l]
                    tid = top_n_list[k]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_rating += 1
                        break

            precision = high_rating / top_n_size
            precisions.append(precision)

        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        avg_precisions.append(avg_precision)
    if top_n_size == 4:
        top_4_results.append(avg_precisions)
    elif top_n_size == 20:
        top_20_results.append(avg_precisions)
    elif top_n_size == 100:
        top_100_results.append(avg_precisions)

In [14]:
for i in top_4_results:
    print(i)
print('------------------')

for i in top_20_results:
    print(i)
print('------------------')

for i in top_100_results:
    print(i)

[0.04357798165137615, 0.052083333333333336, 0.06481481481481481, 0.0755813953488372]
[0.11467889908256881, 0.19791666666666666, 0.20833333333333334, 0.26744186046511625]
------------------
[0.04541284403669725, 0.056944444444444436, 0.05092592592592594, 0.06162790697674418]
[0.09908256880733941, 0.1597222222222222, 0.19537037037037036, 0.21860465116279074]
------------------
[0.04403669724770641, 0.06055555555555556, 0.04666666666666667, 0.06348837209302327]
[0.07633027522935777, 0.11972222222222219, 0.15851851851851848, 0.1934883720930233]
